In [6]:
import concurrent
import threading
import numpy as np
import cython

infile = r"D:\Temp\example.tif"
outfile = r"D:\Temp\example_threaded.tif"
num_workers = 10

%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [15]:
%%cython 
import numpy as np

# cython: boundscheck=False
def compute(double[:, :, :] input):
    """reverses bands inefficiently

    Given input and output uint8 arrays, fakes an CPU-intensive
    computation.
    """
    cdef int I, J, K
    cdef int i, j, k, l
    cdef double val
    I = input.shape[0]
    J = input.shape[1]
    K = input.shape[2]
    print(I, J, K)
    output = np.empty((I, J, K), dtype='double')
    cdef double[:, :, :] output_view = output
    with nogil:
        for i in range(I):
            for j in range(J):
                for k in range(K):
                    val = <double>input[i, j, k]
                    for l in range(2000):
                        val += 1.0
                    val -= 2000.0
                    output_view[~i, j, k] = <double>val
    return output

Content of stdout:
_cython_magic_35f26cc74dbc5b59dd90b9a22e590f0ce1596a4e.c
C:\Users\qingyerichard.zeng\.ipython\cython\_cython_magic_35f26cc74dbc5b59dd90b9a22e590f0ce1596a4e.c(17546): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\qingyerichard.zeng\.ipython\cython\_cython_magic_35f26cc74dbc5b59dd90b9a22e590f0ce1596a4e.c(17555): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\qingyerichard.zeng\.ipython\cython\_cython_magic_35f26cc74dbc5b59dd90b9a22e590f0ce1596a4e.c(17564): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
   Creating library C:\Users\qingyerichard.zeng\.ipython\cython\Users\qingyerichard.zeng\.ipython\cython\_cython_magic_35f26cc74dbc5b59dd90b9a22e590f0ce1596a4e.cp311-win_amd64.lib and object C:\Users\qingyerichard.zeng\.ipython\cython\Users\qingyerichard.zeng\.ipython\cython\_cython_magic_35f26cc74dbc5b59dd90b9a22e590f0ce1596a4e.cp311-win_amd64.exp
Gen

In [18]:
import concurrent.futures
import multiprocessing
import threading

import rasterio
from rasterio._example import compute


def main(infile, outfile, num_workers=10):
    """Process infile block-by-block and write to a new file

    The output is the same as the input, but with band order
    reversed.
    """

    with rasterio.open(infile) as src:

        # Create a destination dataset based on source params. The
        # destination will be tiled, and we'll process the tiles
        # concurrently.
        profile = src.profile
        profile.update(blockxsize=128, blockysize=128, tiled=True)

        with rasterio.open(outfile, "w", **src.profile) as dst:
            windows = [window for ij, window in dst.block_windows()]

            # We cannot write to the same file from multiple threads
            # without causing race conditions. To safely read/write
            # from multiple threads, we use a lock to protect the
            # DatasetReader/Writer
            read_lock = threading.Lock()
            write_lock = threading.Lock()

            def process(window):
                with read_lock:
                    src_array = src.read(window=window)

                # The computation can be performed concurrently
                result = compute(src_array)
                print('process')

                with write_lock:
                    dst.write(result, window=window)

            # We map the process() function over the list of
            # windows.
            with concurrent.futures.ThreadPoolExecutor(
                max_workers=num_workers
            ) as executor:
                executor.map(process, windows)


In [19]:

main(infile, outfile)

Another example for 

In [ ]:
import rasterio
with rasterio.open(r"D:\Temp\example.tif") as src:
    profile = src.profile
    with rasterio.open(r"D:\Temp\example_copy.tif", 'w', **profile) as dst:
        for ji, window in src.block_windows(1):
            r = src.read(1, window=window)
            print(r.shape)
            break